In [6]:
import re
import json
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, RANSACRegressor, HuberRegressor, TheilSenRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

# --- Carga y limpieza ---
with open("AllMolecules_QMProt_Format.json","r") as f:
    text = f.read()
clean = re.sub(r'"n_qubits":\s*0+','"n_qubits": 0', text)
data = json.loads(clean)

# --- DataFrame base ---
records = [
    {"electrons":m["n_electrons"], "qubits":m["n_qubits"], "coefficients":m["n_coefficients"]}
    for m in data["amino_acids"]
    if m["n_electrons"]>0 and m["n_qubits"]>0 and m["n_coefficients"]>0
]
df = pd.DataFrame(records).astype(float)
df["log_e"] = np.log(df["electrons"])
df["log_q"] = np.log(df["qubits"])
df["log_c"] = np.log(df["coefficients"])

# --- Features polinomiales ---
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(df[["log_e"]])
X_lin  = df[["log_e"]].values

# --- Validación cruzada “segura” ---
kf = KFold(5, shuffle=True, random_state=42)
def safe_cv(model,X,y):
    scores = cross_val_score(model,X,y,cv=kf,scoring="r2")
    scores = [s if s>0 else np.nan for s in scores]
    return np.nanmean(scores)

results = []

# --- 1) QUBITS: modelos globales ---
qubit_models = [
    ("OLS",      LinearRegression(),       X_lin),
    ("RANSAC",   RANSACRegressor(LinearRegression(),residual_threshold=1.0,random_state=42), X_lin),
    ("Theil-Sen",TheilSenRegressor(random_state=42), X_lin),
    ("Huber ε=1.35", HuberRegressor(epsilon=1.35, max_iter=1000), X_lin),
    ("Huber+Poly",   HuberRegressor(epsilon=2.0, max_iter=1000), X_poly)
]
for name,model,X_all in qubit_models:
    y = df["log_q"].values
    std,cov = np.std(y), np.std(y)/np.mean(y)*100
    Xtr,Xte,ytr,yte = train_test_split(X_all,y,test_size=0.2,random_state=42)
    model.fit(Xtr,ytr)
    results.append({
      "Target":"Qubits","Scope":"Global","Model":name,
      "Transform":"log","Lambda":"",
      "R² total": round(model.score(X_all,y),3),
      "R² CV":    round(safe_cv(model,X_all,y),3),
      "MAE":       round(mean_absolute_error(yte,model.predict(Xte)),3),
      "RMSE":      round(np.sqrt(mean_squared_error(yte,model.predict(Xte))),3),
      "Std Dev":   round(std,3), "Coef Var (%)": round(cov,2)
    })

# --- 2) QUBITS: segmentos ---
segments_q = [
  ("Small (≤300)",    df[df["electrons"]<=300],       HuberRegressor(epsilon=2.0, max_iter=1000)),
  ("Medium+Large (>300)", df[df["electrons"]>300], TheilSenRegressor(random_state=42))
]
for scope,seg,model in segments_q:
    y = np.log(seg["qubits"]).values
    X = np.log(seg["electrons"]).values.reshape(-1,1)
    std,cov = np.std(y), np.std(y)/np.mean(y)*100
    Xtr,Xte,ytr,yte = train_test_split(X,y,test_size=0.2,random_state=42)
    model.fit(Xtr,ytr)
    results.append({
      "Target":"Qubits","Scope":scope,"Model":model.__class__.__name__,
      "Transform":"log","Lambda":"",
      "R² total": round(model.score(X,y),3),
      "R² CV":    round(safe_cv(model,X,y),3),
      "MAE":       round(mean_absolute_error(yte,model.predict(Xte)),3),
      "RMSE":      round(np.sqrt(mean_squared_error(yte,model.predict(Xte))),3),
      "Std Dev":   round(std,3), "Coef Var (%)": round(cov,2)
    })

# --- 3) COEFFICIENTS: global con Box–Cox + Huber+Poly ---
y_raw = df["coefficients"].values
y_bc,lam = stats.boxcox(y_raw)
std_bc, cov_bc = np.std(y_bc), np.std(y_bc)/np.mean(y_bc)*100
X = X_poly
Xtr,Xte,ytr,yte = train_test_split(X,y_bc,test_size=0.2,random_state=42)
model = HuberRegressor(epsilon=2.0, max_iter=1000).fit(Xtr,ytr)
results.append({
  "Target":"Coefficients","Scope":"Global","Model":"Huber+Poly+BC",
  "Transform":"boxcox","Lambda":round(lam,3),
  "R² total": round(model.score(X,y_bc),3),
  "R² CV":    round(safe_cv(model,X,y_bc),3),
  "MAE":       round(mean_absolute_error(yte,model.predict(Xte)),3),
  "RMSE":      round(np.sqrt(mean_squared_error(yte,model.predict(Xte))),3),
  "Std Dev":   round(std_bc,3), "Coef Var (%)": round(cov_bc,2)
})

# --- 4) COEFFICIENTS: segmentos por cuartiles (Box–Cox+Huber+Poly) ---
quantiles = df["electrons"].quantile([0,0.25,0.5,0.75,1.0]).values
labels = ["Q1","Q2","Q3","Q4"]
df["quartile"] = pd.cut(df["electrons"], bins=quantiles, labels=labels, include_lowest=True)
for q in labels:
    seg = df[df["quartile"]==q]
    if len(seg)<5: continue
    y_raw = seg["coefficients"].values
    y_bc_s,lam_s = stats.boxcox(y_raw)
    X_s = poly.fit_transform(np.log(seg["electrons"]).values.reshape(-1,1))
    std_s, cov_s = np.std(y_bc_s), np.std(y_bc_s)/np.mean(y_bc_s)*100
    Xtr,Xte,ytr,yte = train_test_split(X_s,y_bc_s,test_size=0.2,random_state=42)
    model = HuberRegressor(epsilon=2.0, max_iter=1000).fit(Xtr,ytr)
    results.append({
      "Target":"Coefficients","Scope":q,"Model":"Huber+Poly+BC",
      "Transform":"boxcox","Lambda":round(lam_s,3),
      "R² total": round(model.score(X_s,y_bc_s),3),
      "R² CV":    round(safe_cv(model,X_s,y_bc_s),3),
      "MAE":       round(mean_absolute_error(yte,model.predict(Xte)),3),
      "RMSE":      round(np.sqrt(mean_squared_error(yte,model.predict(Xte))),3),
      "Std Dev":   round(std_s,3), "Coef Var (%)": round(cov_s,2)
    })

# --- Mostrar tabla unificada ---
df_res = pd.DataFrame(results)
#print("Comparativa Unificada Completa", df_res)
df_res.head(20).to_csv("Comparativa_Unificada.csv", index=False)


/Users/parfait.atchade/.pyenv/versions/pennylane_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/var/folders/ns/zbp08xns3tv2_wrm0_mdt5w80000gn/T/ipykernel_88297/882933177.py:38: RuntimeWarning: Mean of empty slice
  return np.nanmean(scores)
/Users/parfait.atchade/.pyenv/versions/pennylane_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [5]:
import re
import json
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import (
    LinearRegression, RANSACRegressor,
    HuberRegressor, TheilSenRegressor
)
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1) Carga y limpieza del JSON
with open("AllMolecules_QMProt_Format.json","r") as f:
    raw = f.read()
clean = re.sub(r'"n_qubits":\s*0+','"n_qubits": 0', raw)
data = json.loads(clean)

# 2) Construcción del DataFrame
records = [
    {"electrons":m["n_electrons"], "qubits":m["n_qubits"], "coefficients":m["n_coefficients"]}
    for m in data["amino_acids"]
    if m["n_electrons"]>0 and m["n_qubits"]>0 and m["n_coefficients"]>0
]
df = pd.DataFrame(records).astype(float)
df["log_e"] = np.log(df["electrons"])
df["log_q"] = np.log(df["qubits"])
df["log_c"] = np.log(df["coefficients"])

# 3) Features polinómicas y CV “segura”
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(df[["log_e"]])
X_lin  = df[["log_e"]].values

kf = KFold(5, shuffle=True, random_state=42)
def safe_cv(model, X, y):
    scores = cross_val_score(model, X, y, cv=kf, scoring="r2")
    return np.nanmean([s if s>0 else np.nan for s in scores])

results = []

# 4) QUBITS – Modelos globales
qubit_models = [
  ("OLS",        LinearRegression(),                       X_lin),
  ("RANSAC",     RANSACRegressor(LinearRegression(),
                                  residual_threshold=1.0,
                                  random_state=42),     X_lin),
  ("Theil-Sen",  TheilSenRegressor(random_state=42),       X_lin),
  ("Huber ε=1.35",HuberRegressor(epsilon=1.35, max_iter=1000), X_lin),
  ("Huber+Poly", HuberRegressor(epsilon=2.0,  max_iter=1000),  X_poly)
]
yq = df["log_q"].values
std_q, cov_q = yq.std(), yq.std()/yq.mean()*100

for name, model, Xall in qubit_models:
    Xtr,Xte,ytr,yte = train_test_split(Xall,yq,test_size=0.2,random_state=42)
    model.fit(Xtr,ytr)
    results.append({
      "Target":"Qubits","Scope":"Global","Model":name,
      "Transform":"log","Lambda":"",
      "R² total":round(model.score(Xall,yq),3),
      "R² CV":   round(safe_cv(model,Xall,yq),3),
      "MAE":      round(mean_absolute_error(yte,model.predict(Xte)),3),
      "RMSE":     round(np.sqrt(mean_squared_error(yte,model.predict(Xte))),3),
      "Std Dev":  round(std_q,3),"Coef Var (%)":round(cov_q,2)
    })

# 5) QUBITS – Segmentos
for scope, segdf, model in [
  ("Small (≤300)",    df[df["electrons"]<=300],   HuberRegressor(epsilon=2.0, max_iter=1000)),
  ("Medium+Large (>300)", df[df["electrons"]>300], TheilSenRegressor(random_state=42))
]:
    yseg = np.log(segdf["qubits"]).values
    Xseg = np.log(segdf["electrons"]).values.reshape(-1,1)
    std_s, cov_s = yseg.std(), yseg.std()/yseg.mean()*100
    Xtr,Xte,ytr,yte = train_test_split(Xseg,yseg,test_size=0.2,random_state=42)
    model.fit(Xtr,ytr)
    results.append({
      "Target":"Qubits","Scope":scope,"Model":model.__class__.__name__,
      "Transform":"log","Lambda":"",
      "R² total":round(model.score(Xseg,yseg),3),
      "R² CV":   round(safe_cv(model,Xseg,yseg),3),
      "MAE":      round(mean_absolute_error(yte,model.predict(Xte)),3),
      "RMSE":     round(np.sqrt(mean_squared_error(yte,model.predict(Xte))),3),
      "Std Dev":  round(std_s,3),"Coef Var (%)":round(cov_s,2)
    })

# 6) COEFFICIENTS – Global con Box–Cox + Huber+Poly
coeff = df["coefficients"].values
ybc, lam_glob = stats.boxcox(coeff)
std_bc, cov_bc = ybc.std(), ybc.std()/ybc.mean()*100
X_c = poly.transform(df[["log_e"]])
Xtr,Xte,ytr,yte = train_test_split(X_c,ybc,test_size=0.2,random_state=42)
mod_c = HuberRegressor(epsilon=2.0, max_iter=1000).fit(Xtr,ytr)
results.append({
  "Target":"Coefficients","Scope":"Global","Model":"Huber+Poly+BC",
  "Transform":"boxcox","Lambda":round(lam_glob,3),
  "R² total":round(mod_c.score(X_c,ybc),3),
  "R² CV":   round(safe_cv(mod_c,X_c,ybc),3),
  "MAE":      round(mean_absolute_error(yte,mod_c.predict(Xte)),3),
  "RMSE":     round(np.sqrt(mean_squared_error(yte,mod_c.predict(Xte))),3),
  "Std Dev":  round(std_bc,3),"Coef Var (%)":round(cov_bc,2)
})

# 7) COEFFICIENTS – Segmentos por cuartiles
quant = df["electrons"].quantile([0,0.25,0.5,0.75,1.0]).values
df["quartile"] = pd.cut(df["electrons"],bins=quant,labels=["Q1","Q2","Q3","Q4"],include_lowest=True)
for q in ["Q1","Q2","Q3","Q4"]:
    seg = df[df["quartile"]==q]
    if len(seg)<5: continue
    raw = seg["coefficients"].values
    ybc_s, lam_s = stats.boxcox(raw)
    X_s = poly.transform(np.log(seg[["electrons"]]).values)
    std_s, cov_s = ybc_s.std(), ybc_s.std()/ybc_s.mean()*100
    Xtr,Xte,ytr,yte = train_test_split(X_s,ybc_s,test_size=0.2,random_state=42)
    mod_s = HuberRegressor(epsilon=2.0, max_iter=1000).fit(Xtr,ytr)
    results.append({
      "Target":"Coefficients","Scope":q,"Model":"Huber+Poly+BC",
      "Transform":"boxcox","Lambda":round(lam_s,3),
      "R² total":round(mod_s.score(X_s,ybc_s),3),
      "R² CV":   round(safe_cv(mod_s,X_s,ybc_s),3),
      "MAE":      round(mean_absolute_error(yte,mod_s.predict(Xte)),3),
      "RMSE":     round(np.sqrt(mean_squared_error(yte,mod_s.predict(Xte))),3),
      "Std Dev":  round(std_s,3),"Coef Var (%)":round(cov_s,2)
    })

# 8) Mostrar resultados
df_res = pd.DataFrame(results).round(3)
print(df_res.to_markdown(index=False))


/Users/parfait.atchade/.pyenv/versions/pennylane_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/Users/parfait.atchade/.pyenv/versions/pennylane_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
/Users/parfait.atchade/.pyenv/versions/pennylane_env/lib/python3.11/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/var/folders/ns/zbp08xns3tv2_wrm0_mdt5w80000gn/T/ipykernel_88297/1950083273.py:39: RuntimeWarning: Mean of empty slice
  return np.nanmean([s if s>0 else np.nan for s in scores])
/Users/parfait.atchade/.pyenv/versions/pennylane_env/lib/python3.11/site-packages/sklearn/utils/validat

ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.